In [2]:
import numpy as np

In [3]:
class Value:
    def __init__(self,value,_children=(),_op=''):
        self.value = value;
        self._prev = set(_children);
        self._op = _op;
        self.grad = 0.0;

    def __repr__(self):
        stringVal = f"{self.value}";
        return stringVal

    def __add__(self,other):
        return Value(self.value + other.value,(self,other),'+');

    def __sub__(self,other):
        return Value(self.value - other.value,(self,other),'-');

    def __mul__(self,other):
        return Value(self.value * other.value,(self,other),'*');

    def __div__(self,other):
        return Value(self.value / other.value,(self,other),'/');

    #TODO: Fungsi dan autograd



In [5]:
n_input_neuron = 3 # jumlah neuron input 
n_input_instance = 1 # jumlah instance input
n_hidden_layer = 1  # jumlah layer
n_output_neuron = 2
n_weight_layers = n_hidden_layer + 1


input_layer_neuron = np.random.randint(5,size=(n_input_instance,n_input_neuron))
output_layer_neuron = np.zeros(n_output_neuron)
hidden_layer_neuron = [0 for i in range(n_hidden_layer)]
weight_layer_neuron = [0 for i in range(n_weight_layers)]

for i in range(n_hidden_layer):
    hidden_layer_neuron[i] = np.zeros(int(input()))

for i in range(n_weight_layers):
    if i==0:
        weight_layer_neuron[i] = np.random.randint(5,size=(n_input_neuron,len(hidden_layer_neuron[i])))
    elif i==n_weight_layers-1:
        weight_layer_neuron[i] = np.random.randint(5,size=(len(hidden_layer_neuron[i-1]),n_output_neuron))
    else:
        weight_layer_neuron[i] = np.random.randint(5,size=(len(hidden_layer_neuron[i-1]),len(hidden_layer_neuron[i])))

print("Input")
print(input_layer_neuron[0])
print("Hidden Layer")
print(hidden_layer_neuron)
print("Output")
print(output_layer_neuron)
print("Weight")
print(weight_layer_neuron[0])

axe = (input_layer_neuron[0]*weight_layer_neuron[0].T)
print("######\n",axe)
print(np.sum(axe,axis=1))

Input
[4 2 3]
Hidden Layer
[array([0., 0., 0., 0.])]
Output
[0. 0.]
Weight
[[3 3 2 3]
 [4 3 4 2]
 [2 1 2 3]]
######
 [[12  8  6]
 [12  6  3]
 [ 8  8  6]
 [12  4  9]]
[26 21 22 25]
